In [27]:
import pickle
import pandas as pd
import numpy as np

In [33]:
with open('pickles/parsed_big_df.pickle', 'rb') as data:
    df = pickle.load(data)

In [61]:
with open('models/best_rfc_fin.pickle', 'rb') as data:
    rfc_fin = pickle.load(data)
    
with open('models/best_rfc_inv.pickle', 'rb') as data:
    rfc_inv = pickle.load(data)

In [5]:
df.head()

,id,url,dateRFC,date,title,intro,text,text_parsed
0,2249,https://vc.ru/tribuna/2249-abrt3,"Tue, 28 Jan 2014 15:46:00 +0400",1390909560,Анализ стартапов от ABRT: Hudway vs. XPages Dy...,Возрождаем забытую практику разбора стартапов ...,Возрождаем забытую практику разбора стартапов ...,анализ стартап abrt hudway xpages dynamic возр...
1,2626,https://vc.ru/flood/2626-riders,"Mon, 27 Jan 2014 11:28:42 +0400",1390807722,RIDERS - глобальное комьюнити экстремалов,Игорь Дебатур и Анатолий Черняков рассказали п...,Игорь Дебатур и Анатолий Черняков рассказали п...,riders глобальный комьюнить экстремалов игорь ...
2,2657,https://vc.ru/flood/2657-evernote,"Tue, 07 Jan 2014 12:31:05 +0400",1389083465,CEO Evernote покаялся и обещает новый дизайн с...,"Фил Либин, основатель и CEO Evernote, признал ...","Фил Либин, основатель и CEO Evernote, признал ...",evernote покаяться обещать новый дизайн сервис...
3,2658,https://vc.ru/flood/2658-nvidia,"Wed, 08 Jan 2014 15:07:20 +0400",1389179240,Nvidia показали стриминг с ПК на телевизор,Nvidia продемонстрировали на пресс-мероприятии...,Nvidia продемонстрировали на пресс-мероприятии...,nvidia показать стриминг телевизор nvidia прод...
4,2660,https://vc.ru/tribuna/2660-startup-tv-series,"Wed, 08 Jan 2014 11:15:49 +0400",1389165349,"Четыре шоу о стартапах, которые вы пропустили",Если вас тоже порядком напрягает истерия вокру...,Если вас тоже порядком напрягает истерия вокру...,четыре стартап который пропустить если тоже по...


In [60]:
# Parameter election
ngram_range = (1, 1)
min_df = 10
max_df = 1.
max_features = 8000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

features = tfidf.fit_transform(df['text_parsed']).toarray()

# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [70]:
tfidf.get_feature_names()

['1000',
 '1200',
 '1500',
 '15px',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2025',
 '2030',
 '20px',
 '2500',
 '2гис',
 '3000',
 '4000',
 '5000',
 '6000',
 'abbyy',
 'absolute',
 'acronis',
 'action',
 'adblock',
 'adidas',
 'adobe',
 'adventum',
 'adwords',
 'after',
 'agency',
 'agile',
 'airbnb',
 'airlines',
 'alexa',
 'alibaba',
 'aliexpress',
 'align',
 'alipay',
 'alphabet',
 'amazon',
 'america',
 'american',
 'amocrm',
 'analytics',
 'andreessen',
 'android',
 'angry',
 'annie',
 'apple',
 'apps',
 'appstore',
 'article',
 'asana',
 'assistant',
 'audi',
 'auto',
 'aviasales',
 'avito',
 'awards',
 'azure',
 'background',
 'badoo',
 'baidu',
 'bank',
 'baring',
 'basecamp',
 'bbdo',
 'be

In [62]:
fin_pred = rfc_fin.predict(features)

In [63]:
inv_pred = rfc_inv.predict(features)

In [64]:
np.unique(fin_pred, return_counts=True)

(array([0, 1], dtype=int64), array([53091,  1731], dtype=int64))

In [65]:
np.unique(inv_pred, return_counts=True)

(array([0, 1], dtype=int64), array([54198,   624], dtype=int64))

In [66]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=8000,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [71]:
for 

id                                                          2668
url                               https://vc.ru/flood/2668-jelly
dateRFC                          Tue, 07 Jan 2014 23:06:25 +0400
date                                                  1389121585
title          Кофаундер Twitter Биз Стоун запустил новую моб...
intro          Один из основателей Twitter Биз Стоун запустил...
text           Один из основателей Twitter Биз Стоун запустил...
text_parsed    кофаундер twitter стоун запустить новый мобиль...
Name: 10, dtype: object
id                                                          2789
url            https://vc.ru/promo/2789-rocketbank-incredible...
dateRFC                          Fri, 11 Apr 2014 16:11:00 +0400
date                                                  1397218260
title                             13 невероятных гифок про банки
intro          Мы уже написали много умных вещей и про то, ка...
text           Мы уже написали много умных вещей и про то, ка...
t

id                                                         10475
url                  https://vc.ru/marketing/10475-tinkoff-quest
dateRFC                          Thu, 10 Sep 2015 14:17:00 +0300
date                                                  1441883820
title          Обзор квеста «Тинькофф Банка»: поездка в Сочи,...
intro          28 июля «Тинькофф Банк» совместно с платежной ...
text           28 июля «Тинькофф Банк» совместно с платежной ...
text_parsed    обзор квеста тинькофф банка поездка сочи побед...
Name: 3889, dtype: object
id                                                         10490
url              https://vc.ru/flood/10490-tinkoff-quest-winners
dateRFC                          Thu, 10 Sep 2015 14:19:05 +0300
date                                                  1441883945
title          Победители «Тинькофф Квеста» получили по 2 252...
intro          Победители конкурса «Тинькофф Квест» с призовы...
text           Победители конкурса «Тинькофф Квест» с призовы...

id                                                          7609
url                                https://vc.ru/promo/7609-citi
dateRFC                          Tue, 14 Apr 2015 14:35:22 +0300
date                                                  1429011322
title          Как выбрать подходящую кредитную карту — обзор...
intro          Представители «Ситибанка» рассказывают, как не...
text           Представители «Ситибанка» рассказывают, как не...
text_parsed    выбрать подходящий кредитный карта обзор ситиб...
Name: 7373, dtype: object
id                                                          7757
url               https://vc.ru/flood/7757-markswebb-rating-2015
dateRFC                          Thu, 16 Apr 2015 09:02:42 +0300
date                                                  1429164162
title          «Тинькофф Банк» снова возглавил рейтинг интерн...
intro          «Тинькофф Банк» второй год подряд возглавил ре...
text           «Тинькофф Банк» второй год подряд возглавил ре...

Name: 10142, dtype: object
id                                                         13314
url                     https://vc.ru/finance/13314-mobile-banks
dateRFC                          Wed, 03 Feb 2016 17:03:22 +0300
date                                                  1454508202
title          Не «Рокетбанком» единым: сравнительный обзор р...
intro          29 января в банке «Интеркоммерц», который обсл...
text           29 января в банке «Интеркоммерц», который обсл...
text_parsed    рокетбанк единый сравнительный обзор российски...
Name: 10176, dtype: object
id                                                         13342
url                          https://vc.ru/social/13342-ok-money
dateRFC                          Tue, 02 Feb 2016 09:24:41 +0300
date                                                  1454394281
title          «Одноклассники» запустили сервис денежных пере...
intro          В «Одноклассниках» появилась возможность перев...
text           В «Одноклассниках» по

Name: 12387, dtype: object
id                                                         15666
url                      https://vc.ru/flood/15666-onlinepay-die
dateRFC                          Sun, 22 May 2016 13:49:56 +0300
date                                                  1463914196
title          «Платежные сервисы в интернете ждёт скорая сме...
intro          Основатель PayQR о перспективах платежных агре...
text           Основатель PayQR о перспективах платежных агре...
text_parsed    платёжный сервис интернет ждать скорый смерть ...
Name: 12416, dtype: object
id                                                         15677
url                       https://vc.ru/flood/15677-alfa-finolog
dateRFC                          Mon, 23 May 2016 12:09:26 +0300
date                                                  1463994566
title          Сервис управленческого учёта «Финолог» позволи...
intro          «Альфа-банк» подключил к своему интернет-банку...
text           «Альфа-банк» подключи

...

Name: 40515, dtype: object
id                                                         61357
url            https://vc.ru/finance/61357-o-regulirovanii-be...
dateRFC                          Fri, 15 Mar 2019 19:27:34 +0300
date                                                  1552667254
title          О регулировании безналичной оплаты в США, иске...
intro          Подборка финтех-новостей за прошедшую неделю о...
text           Подборка финтех-новостей за прошедшую неделю о...
text_parsed    регулирование безналичный оплата иск huawei ра...
Name: 40534, dtype: object
id                                                         61609
url            https://vc.ru/hr/61609-iskusstvennyy-intellekt...
dateRFC                          Tue, 19 Mar 2019 07:58:53 +0300
date                                                  1552971533
title          Искусственный интеллект может заменить финанси...
intro          ИИ меняет рынок труда уже сегодня: обрабатывае...
text           ИИ меняет рынок труда

Name: 43936, dtype: object
id                                                         70038
url            https://vc.ru/finance/70038-elektronnye-platez...
dateRFC                          Fri, 31 May 2019 21:31:07 +0300
date                                                  1559327467
title          Электронные платежные системы: инструкция по п...
intro          Электронные платежные системы необходимы для в...
text           Электронные платежные системы необходимы для в...
text_parsed    электронный платёжный система инструкция приме...
Name: 44088, dtype: object
id                                                         70133
url            https://vc.ru/finance/70133-cb-vmeste-s-bankam...
dateRFC                          Mon, 03 Jun 2019 09:51:57 +0300
date                                                  1559544717
title          ЦБ вместе с банками разработает систему провер...
intro          Банки часто отправляют на почту важные данные,...
text           Банки часто отправляю

id                                                         76235
url            https://vc.ru/finance/76235-karta-s-cash-back-...
dateRFC                          Mon, 22 Jul 2019 11:12:34 +0300
date                                                  1563783154
title                Карта с cash-back – потерять или заработать
intro          Не секрет, что за последнее время появилось до...
text           Не секрет, что за последнее время появилось до...
text_parsed    карта cash back потерять заработать секрет пос...
Name: 46550, dtype: object
id                                                         76302
url            https://vc.ru/finance/76302-kreditnoe-byuro-eq...
dateRFC                          Mon, 22 Jul 2019 15:33:27 +0300
date                                                  1563798807
title          Кредитное бюро Equifax согласилось выплатить д...
intro                Среди пострадавших — более 147 млн человек.
text           Среди пострадавших — более 147 млн человек. Ам..

Name: 49932, dtype: object
id                                                         84673
url            https://vc.ru/finance/84673-tinkoff-vypustil-k...
dateRFC                          Wed, 25 Sep 2019 19:14:19 +0300
date                                                  1569428059
title          «Тинькофф» выпустил карту из белого золота за ...
intro          Сначала карта была выпущена ограниченным тираж...
text           Сначала карта была выпущена ограниченным тираж...
text_parsed    тинькофф выпустить карта белый золото тысяча с...
Name: 49959, dtype: object
id                                                         84737
url            https://vc.ru/finance/84737-rbk-cb-dobavit-opl...
dateRFC                          Thu, 26 Sep 2019 10:38:09 +0300
date                                                  1569483489
title          РБК: ЦБ добавит оплату покупок по биометрии ли...
intro          Технология может стать альтернативой оплате по...
text           Технология может стат

id                                                         93535
url            https://vc.ru/finance/93535-opasnye-svyazi-vo-...
dateRFC                          Thu, 21 Nov 2019 09:26:58 +0300
date                                                  1574317618
title          Опасные связи: во что не стоит вкладывать день...
intro          “Сколько я заработаю?” Именно этот вопрос обыч...
text           “Сколько я заработаю?” Именно этот вопрос обыч...
text_parsed    опасный связь стоить вкладывать деньга начинаю...
Name: 52693, dtype: object
id                                                         93568
url            https://vc.ru/finance/93568-cifrovaya-semibank...
dateRFC                          Wed, 20 Nov 2019 16:29:09 +0300
date                                                  1574256549
title          Цифровая семибанкирщина: самые яркие финансист...
intro          О цифровой трансформации в финансовом секторе ...
text           О цифровой трансформации в финансовом секторе ..